In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import sys
import seaborn as sb
import math
from scipy import interpolate
from scipy import linalg
from numpy import random
from scipy.signal import find_peaks
sys.path.append(r'C:\Users\Joar\Documents\1_Education\NTNU\OneDrive - NTNU\Thesis\Modelling\FD Model')

In [2]:
from FloaterParameters import FloaterParameters
from Environment import Environment
from Buoy import Buoy
from Mass import Mass
from Density import Density
from Area import Area
from GeneralisedCoordinateSystem import GeneralisedCoordinateSystem
from SystemMatrices import MatrixCalculation
from kSolve import ksolve
from OptimizationProblem import OptimizationProblem
from SpectrumStatistics import spectrum_response_statistics, spectrum_response
from jonswap import jonswap
from RAO_Calculation import calulate_RAOs, calculate_column_raos
from SpectrumStatistics import spectrum_response_statistics, spectrum_response, spectrum_response_statistics_single
from InterpolateHydroDResults import InterpolateResults
from InterpolatePanelPressure import interpolate_panel_pressure
from InternalForceCalculation import InternalForces
from plot_code import sub_plots, sub_plots_spectrums
from WindForceSpectrum import WindForceSpectrum
from ReadWadamLis import ReadWadamLis

In [3]:
plt.rcParams['axes.grid'] = True
plt.rcParams["figure.figsize"] = (15, 5)
plt.rcParams.update({'font.size': 15})
plt.rcParams['font.family'] = 'serif' 

In [4]:
# Pulling simulation results
pull = np.array([60,80,12,14])
# Hand Calculations
mufp = FloaterParameters(pull[0], pull[1], pull[2], pull[3])
env = Environment()
rho = Density()
csa = Area(mufp)
buoy = Buoy(mufp, csa, rho)
mass = Mass(mufp, csa, buoy, rho)
coord = GeneralisedCoordinateSystem(mufp, csa, mass, rho, buoy, env)
matrix = MatrixCalculation(coord, mass, mufp, rho, env, csa, buoy)
matrix.stiffness[0, 0] = 1e6
matrix.stiffness[1, 1] = 1e6
matrix.stiffness[5, 5] = 1e9

# Code Switches

In [5]:
run = 1

# Pulling HydroD simulation

In [6]:
path = r'C:\DNVGL\Workspaces\HydroD\fixed_motion\WadamRun1'
reflection = 0
sectional_loads = 1
fixed = 0

In [7]:
file_name = "\\free_motion"

if run == 1:
    file_loc = r'C:\Users\Joar\Documents\1_Education\NTNU\pickle_files'
    file_path = file_loc + file_name
    matrix = MatrixCalculation(coord, mass, mufp, rho, env, csa, buoy)
    hydrod_loads = ReadWadamLis(r'C:\DNVGL\Workspaces\HydroD\free_motion\WadamRun1', sectional_loads, reflection, fixed)
    file_sim = open(file_path, 'wb') 
    pickle.dump(hydrod_loads, file_sim)
    file_sim.close()
else:
    file_loc = r'C:\Users\Joar\Documents\1_Education\NTNU\pickle_files'
    file_path = file_loc + file_name
    infile = open(file_path, 'rb')
    hydrod_loads = pickle.load(infile)
    infile.close()

In [8]:
dof = 2
sec = 0
ylim = 0

In [9]:
hydrod_loads.wave_disc[40,4]

1.5

In [10]:
pd.DataFrame(hydrod_loads.secloads[0,40,0,:,:])

,0,1,2,3,4,5
0,-128118.567043,-1.111703e+06,480107.094983,-7.585197e+05,8.976544e+05,-3.445210e+07
1,13.530530,7.416730e-02,3.251533,-5.416573e-02,3.252011e+00,-5.675307e+05
2,143071.508072,7.496185e+04,190182.532026,1.147965e+05,2.221435e+05,1.859111e+07
3,-12.569909,-9.323634e+01,-16.590417,-9.747788e+01,9.888177e+01,-5.953696e+09
4,-83026.757660,-5.391900e+06,736774.357616,-4.847190e+06,4.902868e+06,-4.860453e+09
5,-135.382960,-5.928366e-01,-25.309481,2.926726e+00,2.547855e+01,1.035893e+10


# Changes to code means no turbine mass, ballasting increased to account for loss of mass

In [11]:
coord.mass_df[['x','y','z', 'Mass [kg]']]

,x,y,z,Mass [kg]
Component,,,,
Front Ballast,0,0.00,-13.798929,1.073863e+06
Back Left Ballast,-60,-40.00,-13.798929,1.073863e+06
Back Right Ballast,-60,40.00,-13.798929,1.073863e+06
Front Column,0,0.00,-0.500000,5.801958e+05
Back Left Column,-60,-40.00,-0.500000,5.801958e+05
Back Right Column,-60,40.00,-0.500000,5.801958e+05
Front Heave Plate,0,0.00,-13.000000,3.810648e+05
Back Left Heave Plate,-60,-40.00,-13.000000,3.810648e+05
Back Right Heave Plate,-60,40.00,-13.000000,3.810648e+05


# Calculate necessary steel thickness for GeniE

In [12]:
h = coord.mass_df.loc['Front Ballast','Mass [kg]']/(np.pi*10**2*8500)
h += 0.07
print(h)

0.4721426470588231
